<a href="https://colab.research.google.com/github/s434/google-colab-projects/blob/main/Amazon_Web_Scrapper/Amazon_Web_Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
from bs4 import BeautifulSoup


In [ ]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 985 kB 6.8 MB/s 
     |████████████████████████████████| 358 kB 50.8 MB/s 
     |████████████████████████████████| 140 kB 57.4 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.


In [ ]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys

In [ ]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
from selenium import webdriver

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=chrome_options)

In [ ]:
url = 'https://www.amazon.ae/'
wd.get(url)

In [ ]:
def get_url(search_term):
  ''' Generate a url from search term '''
  template = "https://www.amazon.ae/s?k={}&crid=1SZIMX3YNKO5S&sprefix=ultrawide+monit%2Caps%2C761&ref=nb_sb_noss_2"
  search_term = search_term.replace(' ', '+')
  return template.format(search_term)

In [ ]:
url = get_url("ultrawide monitor")
print(url)

https://www.amazon.ae/s?k=ultrawide+monitor&crid=1SZIMX3YNKO5S&sprefix=ultrawide+monit%2Caps%2C761&ref=nb_sb_noss_2


In [ ]:
wd.get(url)

we should know how to extract results from next page and even when there are ads and continue extracting results

**EXTRACT THE COLLECTION**

In [ ]:
soup = BeautifulSoup(wd.page_source, 'html.parser')


In [ ]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})

In [ ]:
len(results)

48

**prototype the record**

In [ ]:
item = results[0]

In [ ]:
atag = item.h2.a

In [ ]:
description = atag.text.strip() # strip method to remove the extra spaces

In [ ]:
url = 'https://www.amazon.ae' + atag.get('href')

In [ ]:
price_parent = item.find('span', 'a-price')

In [ ]:
price = price_parent.find('span', 'a-offscreen').text

In [ ]:
rating = item.i.text

In [ ]:
review_count = item.find('span', {'class': 'a-size-base s-underline-text'}).text

In [ ]:
print(review_count)

3


Generalize the pattern to apply to more

In [ ]:
def extract_record(item):
  atag = item.h2.a
  description = atag.text.strip()
  url = 'https://www.amazon.ae' + atag.get('href')

  # price
  try:
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
  except AttributeError:
    return

  # rank and review
  try:
    rating = item.i.text
    review_count = item.find('span', {'class': 'a-size-base s-underline-text'}).text
  except AttributeError:
    rating =''
    review_count = ''

  result = (description, price, rating, review_count, url)
  return result




In [ ]:
records =[]
results = soup.find_all('div', {'data-component-type': 's-search-result'})
for item in results:
  record = extract_record(item)
  if record:
    records.append(record)

In [ ]:
records[0]

('LG 34WP550 34-Inch 21:9 UltraWide Full HD (2560 x 1080) IPS Borderless Monitor with AMD FreeSync - Black',
 'AED\xa01,146.00',
 '5.0 out of 5 stars',
 '3',
 'https://www.amazon.ae/LG-34WP550-34-Inch-UltraWide-Borderless/dp/B09SKMRNK8/ref=sr_1_1?crid=1SZIMX3YNKO5S&keywords=ultrawide+monitor&qid=1664018980&sprefix=ultrawide+monit%2Caps%2C761&sr=8-1')

In [ ]:
for row in records:
  print(row[1])

AED 1,146.00
AED 799.00
AED 1,525.00
AED 1,458.00
AED 1,299.43
AED 1,650.00
AED 1,159.00
AED 2,096.00
AED 1,295.43
AED 4,215.00
AED 1,047.00
AED 2,499.00
AED 1,760.00
AED 1,859.00
AED 1,550.00
AED 1,338.00
AED 649.00
AED 5,519.00
AED 2,999.00
AED 1,489.99
AED 1,201.01
AED 1,669.99
AED 451.00
AED 975.00
AED 1,771.99
AED 2,730.00
AED 745.00
AED 538.00
AED 989.00
AED 2,205.00
AED 2,769.99
AED 3,217.12
AED 1,050.93
AED 2,051.00
AED 1,406.25
AED 1,655.00
AED 4,800.00
AED 1,578.68
AED 1,442.01
AED 2,710.34
AED 855.00
AED 3,499.00
AED 2,849.99
AED 1,897.99
AED 3,909.00


Generating next page ( you can do it using the next button or check the query in the url )




In [ ]:
def get_url(search_term):
  ''' Generate a url from search term '''
  template = "https://www.amazon.ae/s?k={}&crid=1SZIMX3YNKO5S&sprefix=ultrawide+monit%2Caps%2C761&ref=nb_sb_noss_2"

  search_term = search_term.replace(' ', '+')
  #add term query to url
  url = template.format(search_term)

  # add page query placeholder
  url += '&page{}'
  return url

In [ ]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

def get_url(search_term):
  ''' Generate a url from search term '''
  template = "https://www.amazon.ae/s?k={}&crid=1SZIMX3YNKO5S&sprefix=ultrawide+monit%2Caps%2C761&ref=nb_sb_noss_2"

  search_term = search_term.replace(' ', '+')
  #add term query to url
  url = template.format(search_term)

  # add page query placeholder
  url += '&page{}'
  return url

def extract_record(item):
  atag = item.h2.a
  description = atag.text.strip()
  url = 'https://www.amazon.ae' + atag.get('href')

  # price
  try:
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
  except AttributeError:
    return

  # rank and review
  try:
    rating = item.i.text
    review_count = item.find('span', {'class': 'a-size-base s-underline-text'}).text
  except AttributeError:
    rating =''
    review_count = ''

  result = (description, price, rating, review_count, url)
  return result



def main(search_term):
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  wd = webdriver.Chrome('chromedriver',options=chrome_options)

  records =[]
  url = get_url(search_term)

  for page in range(1, 21):
    wd.get(url.format(page))
    soup = BeautifulSoup(wd.page_source, 'html.parser')
    results = soup.find_all('div', {'data-component-type': 's-search-result'})
    for item in results:
      record = extract_record(item)
      if record:
        records.append(record)
  wd.close()

  with open('results.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Description', 'Price', 'Rating' 'ReviewCount', 'Url'])
    writer.writerows(records)







In [ ]:
main('ultrawide monitor')